In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
election = pd.read_csv('/Users/brandonk87/Downloads/MSC 641/Regularization/election2000.csv')
election.head()

,Bush,UnE,Pop,Male,Male18,Pop65,NonMet,Pov,NuHouse,Inc50,Inc75,Inc100
0,56.47,4.6,4447100,48.27,47.26,13.0,32.68,16.1,1472906,14.3,4.3,2.0
1,58.62,6.6,626932,51.69,51.83,5.7,58.48,9.4,85359,36.3,16.8,6.9
2,51.02,3.9,5130632,49.92,49.42,13.0,14.03,13.9,1571330,11.0,4.2,1.8
3,51.31,4.4,2673400,48.80,47.94,14.0,53.66,15.8,836388,10.0,2.9,1.6
4,41.65,4.9,33871648,49.82,49.26,10.6,3.31,14.2,9709296,23.1,10.0,4.4


# Question 1

In [3]:
from sklearn.model_selection import train_test_split
y = election[['Bush']]
x = election.iloc[:,1:]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=13)

/Users/brandonk87/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Question 2

In [4]:
import statsmodels.api as sm
lm = sm.OLS(y_train, sm.add_constant(x_train)).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                   Bush   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.737
Method:                 Least Squares   F-statistic:                     10.93
Date:                Fri, 17 Feb 2023   Prob (F-statistic):           1.87e-07
Time:                        10:29:16   Log-Likelihood:                -118.58
No. Observations:                  40   AIC:                             261.2
Df Residuals:                      28   BIC:                             281.4
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -599.3349    214.124     -2.799      0.0

In [5]:
from sklearn.metrics import mean_squared_error
x_train_predicted = lm.predict(sm.add_constant(x_train))
OLS_MSE_train = mean_squared_error(y_train, x_train_predicted)
OLS_MSE_train

22.0016674800812

In [6]:
x_test_predicted = lm.predict(sm.add_constant(x_test))
OLS_MSE_test = mean_squared_error(y_test, x_test_predicted)
OLS_MSE_test

116.98775766952052

# Question 3

In [7]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
x_train = scale.fit_transform(x_train)
x_test = scale.transform(x_test)

In [8]:
from sklearn.linear_model import Lasso, LassoCV

lassocv = LassoCV(alphas = None, cv = 5, max_iter = 100000)
# creates the lasso function
# using LassoCV, it has cross validation built in
# alpha is None becuase we aren't using a prespecified alpha,
## but are letting the CV find the optimal alpha

lassocv.fit(x_train, y_train)
# applying cv lasso function to the train data

lasso = Lasso(max_iter = 100000)
# creates the new lasso function we will use
lasso.set_params(alpha=lassocv.alpha_)
# makes the lasso function have the optimal lambda value

lasso.fit(x_train, y_train)
# applying optimal lambda lasso function on train data

lasso.coef = pd.Series(lasso.coef_, index=election.iloc[:,1:].columns)
# makes df of the coefficents and their names

print("The optimal lambda is:", lassocv.alpha_)
print(lasso.coef)

/Users/brandonk87/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:1614: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The optimal lambda is: 2.217626713368522
UnE       -0.000000
Pop        0.000000
Male       3.133894
Male18     0.000000
Pop65     -0.000000
NonMet     1.616390
Pov        0.000000
NuHouse    0.000000
Inc50     -0.000000
Inc75     -0.000000
Inc100    -2.888002
dtype: float64


In [9]:
lasso_predicted_train = lasso.predict(x_train)
lasso_MSE_train = mean_squared_error(y_train, lasso_predicted_train)
lasso_MSE_train

48.66377290947794

In [10]:
lasso_predicted_test = lasso.predict(x_test)
lasso_MSE_test = mean_squared_error(y_test, lasso_predicted_test)
lasso_MSE_test

43.528451396211814

# Question 4

In [11]:
from sklearn.linear_model import Ridge, RidgeCV
ridgecv = RidgeCV(cv = 5)
# creates the cross validation ridge function
ridgecv.fit(x_train, y_train)
# applies the cv ridge function to train data
ridge = Ridge(max_iter = 10000)
ridge.set_params(alpha=ridgecv.alpha_)
# creates a new ridge function using the optimal value of lambda
ridge.fit(x_train, y_train)
# applies optimal ridge function to train data 

ridge.coef_ = np.swapaxes(ridge.coef_,0,1)
ridge.coef_ = ridge.coef_.flatten()
# for some reason the ridge function makes a 2D array
# so we need to make it into a 1D for it to work

pd.Series(ridge.coef_, index=election.iloc[:,1:].columns)
# makes a df of the coefficents and their names

UnE       -0.886076
Pop        1.521465
Male       2.367292
Male18     1.479177
Pop65     -1.041412
NonMet     3.460862
Pov        0.027270
NuHouse    0.764344
Inc50     -0.714634
Inc75     -1.070209
Inc100    -2.248464
dtype: float64

In [12]:
print("The optimal lambda is:", ridgecv.alpha_)

The optimal lambda is: 10.0


In [13]:
ridge_predicted_train = ridge.predict(x_train)
ridge_MSE_train = mean_squared_error(y_train, ridge_predicted_train)
ridge_MSE_train

33.78473414498601

In [14]:
ridge_predicted_test = ridge.predict(x_test)
ridge_MSE_test = mean_squared_error(y_test, ridge_predicted_test)
ridge_MSE_test

62.467251845476135

# Question 5

In [15]:
print('OLS MSE Train:',OLS_MSE_train)
print('OLS MSE Test:',OLS_MSE_test)

print('Lasso MSE Train:',lasso_MSE_train)
print('Lasso MSE Test:',lasso_MSE_test)

print('Ridge MSE Train:',ridge_MSE_train)
print('Ridge MSE Test:',ridge_MSE_test)

OLS MSE Train: 22.0016674800812
OLS MSE Test: 116.98775766952052
Lasso MSE Train: 48.66377290947794
Lasso MSE Test: 43.528451396211814
Ridge MSE Train: 33.78473414498601
Ridge MSE Test: 62.467251845476135
